# Synthetic Control Method with regression weight, bootstrapped resample

## Import pandas, numpy, and random

In [1]:
import pandas as pd
import numpy as np
import random
random.seed(12345)

## Read dataset

In [2]:
df = pd.read_excel('Dataset_SCMuzaki.xls', index_col=1)

## Extracting Kanto region data

In [3]:
mat_x1 = np.matrix(df[df.id==48].loc[:,'pop15to64':'n11'].values.tolist()).T
mat_x1  = np.insert(mat_x1,0,1,axis=0)
mat_y1 = np.matrix(df[df.id==48].loc[:,'n12'].values.tolist()).T
iota   = np.matrix(np.ones(30),dtype=float).T

## Create a DataFrame outside the Kanto region.

In [4]:
df_cntl = df[df.intv==0].loc[:,'pop15to64':'n12']

## Extract 30 prefectures from the control pool DataFrame.
Analyze the 30 prefectures extracted as a control group<BR>
Repetition = 1000 times

In [5]:
results_att = []
for x in range(0,1000):
    rand = random.randrange(10000)
    sample_test  = df_cntl.sample(n=30, random_state=rand)
    mat_x0 = np.matrix(sample_test.loc[:,'pop15to64':'n11'].values.tolist()).T
    mat_x0 = np.insert(mat_x0,0,1,axis=0)
    mat_y0 = np.matrix(sample_test.loc[:,'n12'].values.tolist()).T
    
    # Calculate w_reg
    w_reg  = np.dot(np.dot(mat_x0.T,np.linalg.inv(np.dot(mat_x0,mat_x0.T))),mat_x1)
    
    # Check sum_of_weight = 1
    sum_of_weight = np.dot(iota.T,w_reg)
    if sum_of_weight - 1 > 0.001:
        print('Caution! sum_of_weight is not 1')
        continue    
    
    # Check RMSPE = 0
    temp_rmspe = 0
    for i in range(1,13):
        temp = 0
        for j in range(0,30):
            temp += w_reg[j]*mat_x0.T[j,i]
        temp_rmspe += (mat_x1[i]-temp)**2
    rmspe = np.sqrt(temp_rmspe/13)
    if rmspe > 0.0001:
        print('Caution! rmspe > 0.0001', rmspe)
        continue
    
    # Calculate and store ATT 
    syn_cntr_y = np.dot(mat_y0.T,w_reg)
    ATT = mat_y1 - syn_cntr_y
    results_att.append(float(ATT))
    

## Calculate the 2.5th, 50th, and 97.5th percentile of ATT

In [6]:
df_results_att = pd.DataFrame(results_att)
print(df_results_att.quantile([0.025, 0.50, 0.975]))
df_results_att.to_excel('SCM_Wreg_results_att.xlsx', sheet_name='result')

                0
0.025 -157.007322
0.500   83.419375
0.975  324.917967
